In [1]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate 
import pandas as pd
%matplotlib inline

In [2]:
plt.rcParams["figure.figsize"] = (10,10)

We will use the PhotoZDC1 package, specifically photErrorModel, to create the errors, you can grab this package from GitHub with:<br>
git clone https://github.com/LSSTDESC/PhotoZDC1.git <br>
then add the location to your path, as below.  This implements the error model as defined in Ivezic et al (2008) (see: https://arxiv.org/abs/0805.2366 for details)
We have already added a copy to the Photo-z group workspace at NERSC at:
`/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC1/PhotoZDC1/`

In [3]:
#Add the PhotoZDC1 package to the path and import photErrorModel that will be used to generate errors 
#sys.path.insert(0, '/global/u2/s/schmidt9/PhotoZDC1/src')
sys.path.insert(0, '/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC1/PhotoZDC1/src/')
import photErrorModel as errMod

Set up the LSST Error Model parameters.  Alex used some defaults from the Science Book that are not quite the same as the newest numbers from the Science Drivers document (arxiv: 0805.2366v5), so update them here.  We'll use the following number of total visits in 10 years as listed in Table 1:
NVISITS: u: 56  g: 80  r: 184  i: 184  z: 160  y: 160

And check the error model parameters against those listed in Table 2:
Nearly all values will have to be updated

Note that we should use ''theta_eff'', not ''theta'' in the tables!

There is a simple parameter to try to account for extended objects versus the point sources that these models are meant to approximate.  A quick discussion with Zeljko Ivezic and Andy Connolly years ago I believe decided on an offset in m5 value of 0.1 as a very rough approximation, so we'll add that with the errmodel.extendedsource = 0.1 parameter

We can check the param settings with the .\_\_repr\_\_ method

In [4]:
errmodel = errMod.LSSTErrorModel()
errmodel.tvis = 30.0
errmodel.nYrObs=10.0
visits = {'LSST_u':5.6, 'LSST_g':8.0, 'LSST_r':18.4, 'LSST_i':18.4,'LSST_z':16.0,'LSST_y':16.0}
errmodel.nVisYr=visits
msky = {'LSST_u':22.99, 'LSST_g':22.26, 'LSST_r':21.20, 'LSST_i':20.48,'LSST_z':19.60,
        'LSST_y':18.61}
errmodel.msky = msky
#theta = {'LSST_u':0.81, 'LSST_g':0.77, 'LSST_r':0.73, 'LSST_i':0.71,'LSST_z':0.69,'LSST_y':0.68}
theta = {'LSST_u':0.92, 'LSST_g':0.87, 'LSST_r':0.83, 'LSST_i':0.80,'LSST_z':0.78,'LSST_y':0.76}
errmodel.theta = theta
gamma = {'LSST_u':0.038, 'LSST_g':0.039, 'LSST_r':0.039, 'LSST_i':0.039,'LSST_z':0.039,
         'LSST_y':0.039}
errmodel.gamma = gamma
#cm = {'LSST_u':23.09, 'LSST_g':24.42, 'LSST_r':24.44, 'LSST_i':24.32,'LSST_z':24.16,'LSST_y':23.73}
cmshift = {'LSST_u':22.47, 'LSST_g':24.24, 'LSST_r':24.34, 'LSST_i':24.25,'LSST_z':24.11,
           'LSST_y':23.69}
errmodel.Cm = cmshift
km = {'LSST_u':0.491, 'LSST_g':0.213, 'LSST_r':0.126, 'LSST_i':0.096,'LSST_z':0.069,'LSST_y':0.170}
errmodel.km = km
errmodel.airMass = 1.2
errmodel.extendedSource=0.1
errmodel.sigmaSys = 0.005
errmodel.__repr__

<bound method LSSTErrorModel.__repr__ of 
 LSSTErrorModel parameters:
 Exposure time = 30.0 s
 Number of years of observations = 10.0
 Number of visits per year per band: {'LSST_u': 5.6, 'LSST_g': 8.0, 'LSST_r': 18.4, 'LSST_i': 18.4, 'LSST_z': 16.0, 'LSST_y': 16.0}
 Systematic error = 0.005 mag
 Airmass = 1.2
 Sky brightness per band: {'LSST_u': 22.99, 'LSST_g': 22.26, 'LSST_r': 21.2, 'LSST_i': 20.48, 'LSST_z': 19.6, 'LSST_y': 18.61} (mag)
 Seeing per band: {'LSST_u': 22.99, 'LSST_g': 22.26, 'LSST_r': 21.2, 'LSST_i': 20.48, 'LSST_z': 19.6, 'LSST_y': 18.61} (arcsec)
 gamma per band: {'LSST_u': 0.038, 'LSST_g': 0.039, 'LSST_r': 0.039, 'LSST_i': 0.039, 'LSST_z': 0.039, 'LSST_y': 0.039}
 Cm per band: {'LSST_u': 22.47, 'LSST_g': 24.24, 'LSST_r': 24.34, 'LSST_i': 24.25, 'LSST_z': 24.11, 'LSST_y': 23.69}
 Extinction coeff. per band: {'LSST_u': 0.491, 'LSST_g': 0.213, 'LSST_r': 0.126, 'LSST_i': 0.096, 'LSST_z': 0.069, 'LSST_y': 0.17}
 Extended source model: add 0.1 mag to 5-sigma depth for poi

Let's check the 5 sigma point source depths, which should equal the following numbers at Zenith (we need to account for the $\Delta Cm$ values which account for instrumental noise, listed below are m5 values minus the $\Delta Cm^{ \infty}$ values in Table 2 of the Ivezic paper)<br>
u: 23.78 - 0.62 = 23.16<br>
g: 24.81 - 0.18 = 24.63<br>
r: 24.35 - 0.10 = 24.25<br>
i: 23.92 - 0.07 = 23.85<br>
z: 23.34 - 0.05 = 23.29<br>
y: 22.45 - 0.04 = 22.41<br>

In [5]:
filts = ('LSST_u','LSST_g','LSST_r','LSST_i','LSST_z','LSST_y')
for filt in filts:
    print ("%s  %.2f"%(filt,errmodel._m5PointSources(cmshift[filt],msky[filt],theta[filt],30.,
                                                     km[filt],1.0)))
    #args are Cm, msky, theta, tvis (30 seconds), km, and X (airmass, set to zenith so 1.0)

LSST_u  23.17
LSST_g  24.63
LSST_r  24.26
LSST_i  23.85
LSST_z  23.29
LSST_y  22.41


m5 values check out to the .01 level, so that is good!<br>


We will test on cosmoDC2v1.1.4_small, which has 51 hdf5 files for 17 healpix pixels broken into 3 redshift ranges.  When things are finalized we can switch to cosmoDC2_v1.1.4_image for the larger catalog

In [6]:
import GCRCatalogs
## check version
print('GCRCatalogs =', GCRCatalogs.__version__, '|' ,'GCR =', GCRCatalogs.GCR.__version__)

GCRCatalogs = 0.10.0 | GCR = 0.8.7


In [7]:
#use this to print all available catalogs, note the "False" to actually list them all
#print('\n'.join(sorted(GCRCatalogs.get_available_catalogs(False))))

In [8]:
#gc = GCRCatalogs.load_catalog('proto-dc2_v5.0')
gc = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small')

In [9]:
#List all available quantities in the catalog
#print('\n'.join(sorted(gc.list_all_native_quantities())[:]))

Set up some names that we will use to grab the six LSST magnitudes from the catalog reader

In [10]:
filts = ('u','g','r','i','z','y')
short_mag_cols = []
mag_cols = []
for filt in filts:
    mag_cols.append('LSST_filters/magnitude:LSST_%s:observed:dustAtlas'%filt)
    short_mag_cols.append('lsst_%s'%filt)
all_cols = mag_cols
column_dict = dict(zip(mag_cols,short_mag_cols))
all_cols.append('redshift')
all_cols.append('baseDC2/galaxy_id')
print (filts,mag_cols)


('u', 'g', 'r', 'i', 'z', 'y') ['LSST_filters/magnitude:LSST_u:observed:dustAtlas', 'LSST_filters/magnitude:LSST_g:observed:dustAtlas', 'LSST_filters/magnitude:LSST_r:observed:dustAtlas', 'LSST_filters/magnitude:LSST_i:observed:dustAtlas', 'LSST_filters/magnitude:LSST_z:observed:dustAtlas', 'LSST_filters/magnitude:LSST_y:observed:dustAtlas', 'redshift', 'baseDC2/galaxy_id']


We will be computing the model errors for our mock galaxies.  The error fuction produced by the PZDC1.getMagError function is very smooth.  Rather than computing this for every galaxy, we will compute a grid for each filter and interpolate between them to speed up the computation.  We will set an ad hoc upper limit of 35 for magnitude, above this magnitude we will set the spline to return 99.0.  This should only really occur for flux <= 0.0 and a NaN in the log(flux).

In [11]:
mag_min = 10.
mag_max = 45.
num_gridpts = 4001 #parameters for magerr_spline
mag_grid = np.linspace(mag_min,mag_max,num_gridpts)
#print (mag_grid)

Now, define the functions for making the spline and creating new magnitudes.  For scatterMag, we need to convert to flux, add Gaussian uncertainty, then convert back to flux.

In [12]:
def makeSpline(grid,filt):
    fx =  [errmodel.getMagError(x,filt) for x in grid]
    return fx

In [13]:
def scatterMag(mag,mag_err):
    """Add proper uncertainty to magnitude
    Parmeters
    ---------
    mag: 'float' array-like
        true magnitudes 
    magerr: 'float' array-like
        magnitude uncertainties 
    Returns
    -------
    new_mag: 'float' array-like
        magnitudes with proper uncertainty, or 99 if flux was negative after error added
    """
    floor = 1.e-14 # set a minimum flux value, corresponding to mag = 35 for now
    floor_mag = -2.5*np.log10(floor)
    flux = np.power(10.,-0.4*mag) 
    err_flux = np.multiply(flux,mag_err)*0.4*np.log(10.)
    mask = (err_flux<0.0) #check for objects that have scattered to negative flux
    err_flux[mask] = 0.005
    tmperr = np.random.normal(0.0,err_flux)
    new_flux = np.subtract(flux,tmperr)
    mask = (new_flux < floor)
    new_flux[mask] = floor
    new_mag = -2.5*np.log10(new_flux)
    mask2 = np.logical_or(new_mag != new_mag,new_mag>=floor_mag)#error for NaN or Inf, which can happen if flux is negative or 0
    new_mag[mask2] = 99.0
    print ("Number of non-detections: %d"%(np.sum(mask2)))
    return new_mag

Set up an iterator to grab each healpix pixel, and also grab the native filenames.  We will create analagous filenames for our magnitues+errors and new uncertainties.

In [14]:
#print ("number of objects in catalog: %d"%(len(data['redshift'])))
gc_iterator = gc.get_quantities(all_cols,return_iterator=True)
fpath_iterator= gc._healpix_files.values()

In [15]:
base_output_path = "/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/10_year_error_estimates/"
pz_filenames = []
for tmppath in fpath_iterator:
    filename = os.path.split(tmppath)[-1]
    basename = os.path.splitext(filename)[0]
    pz_filenames.append(base_output_path+basename+"_magwerr.h5")
print ('\n'.join(pz_filenames))

/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/10_year_error_estimates/z_0_1.step_all.healpix_10070_magwerr.h5
/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/10_year_error_estimates/z_0_1.step_all.healpix_10071_magwerr.h5
/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/10_year_error_estimates/z_0_1.step_all.healpix_10072_magwerr.h5
/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/10_year_error_estimates/z_0_1.step_all.healpix_10198_magwerr.h5
/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/10_year_error_estimates/z_0_1.step_all.healpix_10199_magwerr.h5
/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/10_year_error_estimates/z_0_1.step_all.healpix_10200_magwerr.h5
/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/10_year_error_estimates/z_0_1.step_all.healpix_10326_magwerr.h5
/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/

In [16]:
print(len(pz_filenames))

51


Next, for each filter, compute magnitude errors, add the uncertainties to the magnitudes, and re-calculate the magnitude errors.  Then add these new "scattered" (in flux) quantities to the data frame, and write out to hdf5 files. 

We will stoer them on cori in the file path listed above.

In [17]:
%%time
pd.set_option('precision', 5)
cols_to_write=['redshift','baseDC2/galaxy_id']
for filt in filts:
    cols_to_write.append("scatmag_%s"%filt)
    cols_to_write.append("scaterr_%s"%filt)
#for ii,data in enumerate(gc_iterator):
#    tmpfile = "/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/10_year_error_estimates/%d.hdf5"%ii
for data,tmpfile in zip(gc_iterator,pz_filenames):
#data = gc.get_quantities(all_cols)
    df = pd.DataFrame(data, columns = all_cols)
    for filt,magcol in zip(filts,mag_cols):
        print ("working on filter %s"%(filt))
        tmpmag = data[magcol]
        grid_errs = makeSpline(mag_grid,'LSST_%s'%filt)
        #print (grid_errs)
        errspline = scipy.interpolate.interp1d(mag_grid,grid_errs,kind='linear',bounds_error=False,
                                               fill_value=99.0)#return 99.0 outside interval
        newerrx = errspline(tmpmag)
        newmag = np.array(scatterMag(tmpmag,newerrx),dtype=np.float32)
        #we need to recalculate the magnitude error based on the new magnitude    
        newerr = np.array(errspline(newmag),dtype=np.float32)  
        magname = 'scatmag_%s'%filt
        errname = 'scaterr_%s'%filt
        df[magname] = newmag
        df[errname] = newerr
    smalldf = df[cols_to_write]
    smalldf.to_hdf(tmpfile, 'df',mode='w')

working on filter u
Number of non-detections: 875228
working on filter g
Number of non-detections: 386966
working on filter r
Number of non-detections: 180324
working on filter i
Number of non-detections: 119891
working on filter z
Number of non-detections: 189083
working on filter y
Number of non-detections: 395601
working on filter u
Number of non-detections: 876932
working on filter g
Number of non-detections: 387515
working on filter r
Number of non-detections: 180709
working on filter i
Number of non-detections: 119702
working on filter z
Number of non-detections: 188938
working on filter y
Number of non-detections: 395152
working on filter u
Number of non-detections: 874851
working on filter g
Number of non-detections: 386790
working on filter r
Number of non-detections: 180610
working on filter i
Number of non-detections: 120009
working on filter z
Number of non-detections: 189051
working on filter y
Number of non-detections: 395202
working on filter u
Number of non-detections: 